# RateML

### Using RateML to generate a CUDA model file and Pyunicore to excute parameter sweeps on HPC (JUSUF cluster)

In [ ]:
!git clone https://github.com/DeLaVlag/tvb-root.git

In [ ]:
# install tvb-library
%cd tvb-root/scientific_library/
!python setup.py install
# install tvb-bin
%cd ../tvb_bin
!python setup.py install
# install tvb-data
! pip install tvb-data
%cd ../../

## Building a model
Building rate based models in RateML, start by creating an XML model file. To understand which constructs can be used to build the model, one should take a closer look at the README file. The cell below will prints the latest README file from the repository. Every construct which can be used, is explained.

In [5]:
from IPython.display import Markdown, display, Code
mdfile = open("tvb-root/scientific_library/tvb/rateML/README.md","r")
model = mdfile.read()
display(Markdown(model))
mdfile.close()

## Rate based model generation for Python TVB or CUDA models
This readme describes the usage of the code generation for models defined in LEMS based XML to Cuda or Python.
The pyLEMS framework is used and will be installed with TVB. 
XML2model.py holds the class which start the templating.
To use RateML, create an object of the RateML class in which the arguments: the model filename, language, XML file location and model output location, are entered.
```python
RateML('model_filename', language=('python' | 'cuda'), 'path/to/your/XMLmodels', 'path/to/your/generatedModels')
```
The model filename is the name of XML file which will also be the name of the model class for Python models and the kernel name for CUDA models. 
The language (python | cuda) determines if either a python or cuda model will be generated. 
It expects a model_filename.xml to be present in 'path/to/your/XMLmodels' for either Python or CUDA model generation.
The generated file will be placed in ''path/to/your/generatedModels'.
The produced filename is a lower cased model_filename.c or -.py which contains a kernel called model_filename or a class named Model_filenameT.

    .. moduleauthor:: Michiel. A. van der Vlag <m.van.der.vlag@fz-juelich.de>
    .. moduleauthor:: Marmaduke Woodman <marmaduke.woodman@univ-amu.fr>
    .. moduleauthor:: Sandra Diaz <s.diaz@fz-juelich.de>

## XML template
The XML template below can be found in /XMLmodels and can be used to start the construction of a model. This section describes each of the constructs need to define the model.
Both CUDA and Python can be generated from the same XML model file, however the CUDA variant would need componenttypes for coupling and noise. 
The order of constructs in this figure denotes the sequences in which they need to appear in the XML file. The Pylems expression parser, which checks for mathematical correctness, is active on all the fields which hold expressions. The power symbol in the expression fields needs to be entered within curly brackets: {x^2}, because it needs to be translated to the specific power operator for each language.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<Lems description="Rate based generic template">

```

### Derivatives
To define the time derivatives of the model, a component type with the name: “derivatives”, should be created. This component type can hold other constructs which define the initialization and dynamics definition of the model. 
```xml
    <ComponentType name="derivatives">
```
\
The **Parameter** construct defines variables to be used in a parameter sweep. The name keyword specifies the name and the dimension keyword specifies the type of the parameter. Parameter is only supported by CUDA. rateML/run/__main__.py holds an example of coupling and speed exploration. In this example the coupling parameters have the range: logspace(1.6, 3.0, coupling_runtime_dimension) and speed parameters have the range logspace(1.6, 3.0, speed_runtime_dimension)
```xml
        <Parameter name="[p_name0]" dimension='[type=float]'/>
        <Parameter name="[p_name1]" dimension='[type=float]'/>
```
\
The **DerivedParameter** construct specifies parameters which are derived during run-time of the model in dependence of the parameters used in sweeps. This construct is only supported by the CUDA models and will generate a float with the const keyword, which is initialized with the expression. Special derived parameters are “rec_speed_dt” and “nsig”. If “rec_speed_dt” is not defined the coupling will not have a temporal aspect, meaning that the delay between nodes will be set to 0. If the added noise needs to be amplified the keyword “nsig” should be used. See coupling and noise sections for more information.
```xml
        <DerivedParameter name="[name]" expression="[expression]"/>
```
\
The **Constant** construct is used to create constants of the type float. The value field represents the initial value and a description can be entered optionally in the description field. (TODO. Description bug in LEMS: The description attibute is mapped to the symbol attribute in LEMS.py. For now no description field does not translate to description field in Python generations). For the Python models a domain must be specified, this can be done in the dimension field. Either the domain in the format as listed is entered, or None is entered to exclude it. This domain is used to generate GUI slider bars and value validation in the TVB Framework. For the CUDA models, the dimension field can be omitted.
```xml
        <Constant name="[name]" domain="lo=[value], hi=[value], step=[value]" default="[value]" description="[optional]"/>
```
\
The **Exposures** construct specifies variables that the user wants to monitor and that are returned by the computing kernel: name is the name of the variable to be monitored, the keyword dimension defines any mathematical operation performed on a variable of interest. 
```xml
        <Exposure name="[name]" dimension="[Expression]"/>
```
\
The **StateVariable** construct is used to define the state variables of the model. The state variables in a Python TVB model are initialized with a numpy.array with the values entered in the dimension field. This field can be omitted in the CUDA models, in which state variables are single variables initialized to 0. The exposure field sets the state variables' upper and lower boundaries, for both languages. These boundaries ensure that the values of state variables stay within the set boundaries and resets the state if the boundaries are exceeded.
```xml
        <Dynamics>
            <StateVariable name="[name]" dimension="[range]" exposure="[lower_bound], [upper_bound]"/>
```
\
The **DerivedVariable** construct can be used to define temporary variables which are the building blocks of the time derivatives. Any formula can be used, and the syntax is: name=value.
```xml
            <DerivedVariable name="[name]" value="[expression]" />
```
\
The **ConditionalDerivedVariable** construct can be used to create if-else constructs. Within the construct a number of Cases can be defined to indicate what action is valid when the condition is true of false. The condition field holds the condition for the action. The escape sequences '\&lt(=);' and '\&gt;(=)' for less- or greater then (or equal to) are used to specify the condition. The value field holds the expression of which the value will be assigned to the variable indicated with the name field. At least one case should be defined. Two cases will produce an if-else structure. There is no need to enter the ‘else’ keyword as is shown in the example, this will be produced automatically. More than two cases will result in the usage of ‘else-if’ statements.
```xml
            <!--if [condition]: [name] = [expression1]
                else if [condition]: [name] = [expression2] 
                else: [expression3] -->
            <ConditionalDerivedVariable name="[name]">
                <Case condition="[variable] [conditional operator] [value]" value="[expression1]"/>
                <Case condition="[variable] [conditional operator] [value]" value="[expression2]"/>
                <Case condition="" value="[expression3]"/>
            </ConditionalDerivedVariable>
```
\
The **TimeDerivative** construct defines the model dynamics in terms of  derivative functions. The variable field specifies the variable on the left-hand side of the differential equation, while the value field specifies the right-hand side expression. 
```xml
            <TimeDerivative variable="[name]" value="[expression]"/>
        </Dynamics>
    </ComponentType>
```

### Coupling
To define **coupling functions**, which is only applicable for CUDA models, the user can create a coupling component type.
To identify this coupling component type, the name field should include the phrase coupling. It will construct a double for loop in which for every node, every other node's state is fetched according to connection delay, multiplied by connection weight.
The Python models have coupling functionality defined in a separate class which contains pre-defined functions for pre- and post synaptic behavior. With RateML it is not possible to define the coupling function for the Python models. For the python models, a number of hardcoded c_pop variables are defined, which can be used to implement the coupling value per population. Each of these variables define the coupling for each neuron population defined.
```xml
    <ComponentType name="coupling_[name]">
```
\
The coupling **Parameter** construct specifies the variable that stores intermediate results from computing coupling inputs and its dimension field indicates which state variable the coupling computation should consider and which is fetched from memory. For instance, in case of the Epileptor model, six state variables are present and either one of them could (theoretically) be considered in computing the coupling. This function has been added to give the user more freedom in defining the coupling function. The dimension param indicates the nth defined statevariable of which values are fetched from memory.
The parameter name should appear in the user defined 'pre' or 'post' coupling function, if user want temporal model aspects.
```xml
        <Parameter name="[name]" dimension='[0-n]'/>
```
\
The coupling **DerivedParameter** construct defines the name of the variable to store the final coupling result and the value field contains a function that transforms the sums of all inputs to a specific node. This variable, 'dp_name', can be used to incorporate the coupling value in the dynamics of the model and is eventually used to 'export' the coupling value out of the for loops. This value should then be used in time derivative definition 
Syntax: [name] *= [expression].
```xml
        <DerivedParameter name="[dp_name]" value="[expression]"/>
```
\
The **DerivedVariable** construct can be used to enter a 'pre'- and 'post'-synaptic coupling function. The final result is that the pre- is multiplied with the post synaptic coupling function and again multiplied with the weights of the node. This result is assigned to 'dp_name', which is shown on line 16 of the listing below. If the derivatives component type does not have a derived parameter named 'rec_speed_dt' the coupling function will not have the delay aspect and thus does not have a temporal component. The listing below also shows the fetching of the delays and the usage of 'rec_speed_dt'. If 'rec_speed_dt' is not defined 'dij' will be set to 0. Other variables or constants defined inside or outside of this component type, can also be used to populate this function.
```xml
        <Dynamics>
            <!-- Construct to define the pre and post coupling function
                Name should always be 'pre' and 'post'
                [dp_name] += node_weight * [pre.expression] * [post.expression] -->
            <DerivedVariable name="[pre]" value="[expression]"/>
            <DerivedVariable name="[post]" value="[expression]"/>
        </Dynamics>
    </ComponentType>

    <!-- It is possible to decribe a 2nd coupling funtion, with the same as the syntax as previous. -->
    <ComponentType name="coupling_function_pop2">
            
        <!-- etc... -->
    </ComponentType>
```

### Noise
To specify **noise addition** to the model dynamics, a component type with the name 'noise' can be defined, which is only applicable for CUDA models. The CUDA models make use of the Curand library to add a random value to the calculated derivatives. As is mentioned in the derivatives section, if the derivatives component type has a derived parameter with the name 'nsig' a noise amplification of value will be used to amplify the noise. 
```xml
    <!--Type noise for noise. Only for CUDA models. 
        Will add curand noise to state variable and amplify with constant 'nsig' if defined.
        [state_var_name] += nsig * curand_normal(&crndst) + [time_der_var] -->
    <ComponentType name="noise"/>


</Lems>
```

```c
// Loop over nodes
for (int i_node = 0; i_node < n_node; i_node++){
    c_pop1 = 0.0f;
    V = state((t) % nh, i_node + 0 * n_node);
  unsigned int i_n = i_node * n_node
     // Loop over nodes
     for (unsigned int j_node = 0; j_node < n_node; j_node++){
       // Get the weight of the coupling between node i and node j
       float wij = weights[i_n + j_node];
       if (wij == 0.0) continue;
       // Get the delay between node i and node j
       unsigned int dij = lengths[i_n + j_node] * rec_speed_dt;
       // Get the state of node j which is delayed by dij
       float V_j = state(((t - dij + nh) % nh), j_node + 0 * n_node);
       // Sum using coupling function (constant * weight * pre * post)
       c_pop1 += wij * c_c_a * 2 * sin(V_j - V);
    }
}
// Export c_pop1 outside loop and process
c_pop1 *= global_coupling;
// Do derivative calculation
...
```
*Listing 1. Generated CUDA code for the coupling functionality*

### Numba and Python coupling
In RateML the dynamics of the Python rate models are integrated using Numba vectorization (Lam et al., 2015). They are mapped to a generalized universal function (gufuncs) using Numba's guvectorize decorator to compile a pure Python function directly into machine code that operates over NumPy arrays, as fast as a C implementation (Numba, “Creating NumPy universal functions.”, https://numba.pydata.org/numba-doc/latest/user/vectorize.html). An example of a Numba generated guvectorize function is displayed in Listing 2. In this example the gufunc *_numba_dfun_Epileptor* accepts two n and m sized float64 arrays and 19 scalars as input and returns a n sized float64 array. The benefit of writing Numpy gufuncs with Numba's decorators, is that it automatically uses features such as reduction, accumulation and broadcasting to efficiently implement the algorithm in question.
The Python models have coupling functionality defined in a separate class which contains pre-defined functions for pre- and post synaptic behavior. With RateML it is not possible to define the coupling function for the Python models. For these models, as is shown in listing 2, a number of hardcoded c_pop*n* variables are defined, which can be used to implement the coupling value per population. Each of these variables define the coupling for each neuron population defined.
```python
@guvectorize([(float64[:], float64[:], (float64 * 19), float64[:])], 
    '(n),(m)' + ',()'*19 + '->(n)', nopython=True)
def _numba_dfun_EpileptorT(vw, coupling, a, b, c, d, r, s, x0, Iext, slope, 
    Iext2, tau, aa, bb, Kvf, Kf, Ks, tt, modification, local_coupling, dx):

    c_pop1 = coupling[0]
    c_pop2 = coupling[1]
    c_pop3 = coupling[2]
    c_pop4 = coupling[3]
    ... # calculate derivatives
    
    return dx
```
*Listing 2. Generated CUDA code for the coupling functionality*

## Files in ~/rateML/
* XML2model.py   		    : python script for initiating model code generation of either CUDA or Python models
* tmpl8_cuda.py 		    : Mako template converting XML to CUDA
* tmpl8_python.py 		    : Mako template converting XML to Python
* /XMLmodels                : folder containing LEMS based XML model example files as well as a model template
* /generatedModels          : resulting model folder
* /run/                     : folder with example script to run generated model
* /run/__main__.py          : cuda setup file
* /run/cuda_run.py          : cuda run file using Pycuda


## Requirements
* Mako templating
* Pycuda
* pyLEMS


## Running an example from ~/run folder
In the folder 'tvb/rateML/run/' an example can be found on how to run the model generator and the CUDA model on a GPU. From this folder, execute \_\_main\_\_.py on a CUDA enabled machine: 
```bash 
python ./__main__.py --model [model_filename] -c [coupling dimension] -s [speed dimension] 
```

The block dimensions default to 32x32 which has shown to have the best occupancy, the grid is adjusted accordingly. In the output.out a small report on the success of the simulation is printed. 

## Generate a model
After reading the README, one should be able to build an XML model file. Lets use the relatively small Kuramoto model as an example. 
Your model should look like similar to the Kuramoto python file and define some constants, an exposure and dynamics behavior. The dynamics for the Kuramoto consist of a state variable, a derived variable and a time derivative. Except for the derived variable, there are the construct that a RateML XML model file should contain.

In [8]:
# Open the Kuramoto model
model_location = "tvb-root/scientific_library/tvb/rateML/XMLmodels/kuramoto.xml"
xmlfile = open(model_location,"r")
model = xmlfile.read()
display(Markdown(model))
xmlfile.close()

<Lems description="Kuramoto Base type of any cell/population which has a (dimensionless) rate _R.">

    <ComponentType name="derivatives">

        <!-- Parameters will be assumed to be swept. sweep data will be matched by order of entry -->
        <Parameter name="global_speed" dimension='float'/>
        <Parameter name="global_coupling" dimension='float'/>

        <DerivedParameter name="rec_n" value="1.0f / n_node"/>
        <DerivedParameter name="rec_speed_dt" value="1.0f / global_speed / (dt)"/>
        <DerivedParameter name="nsig" value="sqrt(dt) * sqrt(2.0 * 1e-5)"/>

        <Constant name="omega" value="60.0 * 2.0 * 3.1415927 / 1e3" dimension=""  
                  description="sets Kuramoto oscillator base line frequency [rad/ms]"/>

        <Exposure name="theta" dimension="sin(theta)" />

        <Dynamics>
            <!-- Use PI to select the PI wrapper (ie Kuramoto) -->
            <StateVariable name="theta" dimension="0, PI * 2"/>

            <TimeDerivative variable="dtheta" value="omega + c_pop1"/>
        </Dynamics>
    </ComponentType>

    <ComponentType name="coupling_function">

        <Parameter name="V_j" dimension='0'/>
        <DerivedParameter name="c_pop1" value="global_coupling * rec_n"/>

        <Constant name="a" value="1" dimension="" description="Rescales the connection strength."/>

        <Dynamics>
            <DerivedVariable name="pre" value="sin(V_j - V)"/>
            <DerivedVariable name="post" value="a"/>
        </Dynamics>
    </ComponentType>

    <ComponentType name="noise"/>
</Lems>


## Generating the model code

We will call the templating function in order to automatically generate the model code.
In XML2model.py the class 
```python
RateML('model_filename', language=('python' | 'cuda'), 'path/to/your/XMLmodels', 'path/to/your/generatedModels')
``` 
will start the code generation.

In [12]:
from tvb.rateML.XML2model import RateML
# Start generation

# some preexisting examples:
# model_filename = 'montbrio'
# model_filename = 'oscillator'
# model_filename = 'kuramoto'
# model_filename = 'rwongwang'
# model_filename = 'epileptor'
model_filename = "kuramoto"
language = "cuda"
XMLin = "tvb-root/scientific_library/tvb/rateML/XMLmodels"
GenModOut = "tvb-root/scientific_library/tvb/rateML/generatedModels"
RateML(model_filename, language, XMLin, GenModOut)

2020-11-18 16:52:26,210 - INFO - tvb.rateML.XML2model - True validation of tvb-root/scientific_library/tvb/rateML/XMLmodels/kuramoto.xml against https://raw.githubusercontent.com/DeLaVlag/tvb-root/xsdvalidation/scientific_library/tvb/rateML/rML_v0.xsd


In [13]:
# Open the generated python Kuramoto model
model_location = "tvb-root/scientific_library/tvb/rateML/generatedModels/kuramoto.c"
genModFile = open(model_location,"r")
model = genModFile.read()
display(Code(model, language='c'))
# display(Markdown(model))
xmlfile.close()

#include <stdio.h> // for printf
#define PI_2 (2 * M_PI_F)

// buffer length defaults to the argument to the integrate kernel
// but if it's known at compile time, it can be provided which allows
// compiler to change i%n to i&(n-1) if n is a power of two.
#ifndef NH
#define NH nh
#endif

#ifndef WARP_SIZE
#define WARP_SIZE 32
#endif

#include <curand_kernel.h>
#include <curand.h>
#include <stdbool.h>

__device__ float wrap_it_PI(float x)
{
    bool neg_mask = x < 0.0f;
    bool pos_mask = !neg_mask;
    // fmodf diverges 51% of time
    float pos_val = fmodf(x, PI_2);
    float neg_val = PI_2 - fmodf(-x, PI_2);
    return neg_mask * neg_val + pos_mask * pos_val;
}
__device__ float wrap_it_theta(float theta)
{
    float thetadim[] = {None};
    if (theta < thetadim[0]) theta = thetadim[0];
    else if (theta > thetadim[1]) theta = thetadim[1];

    return theta;
}

__global__ void KuramotoT(

        // config
        unsigned int i_step, unsigned int n_node, unsigned int nh, unsigned int n_step, unsigned int n_params,
        float dt, float speed, float * __restrict__ weights, float * __restrict__ lengths,
        float * __restrict__ params_pwi, // pwi: per work item
        // state
        float * __restrict__ state_pwi,
        // outputs
        float * __restrict__ tavg_pwi
        )
{
    // work id & size
    const unsigned int id = (gridDim.x * blockDim.x * threadIdx.y) + threadIdx.x;
    const unsigned int size = blockDim.x * blockDim.y * gridDim.x * gridDim.y;

#define params(i_par) (params_pwi[(size * (i_par)) + id])
#define state(time, i_node) (state_pwi[((time) * 1 * n_node + (i_node))*size + id])
#define tavg(i_node) (tavg_pwi[((i_node) * size) + id])

    // unpack params
    // These are the two parameters which are usually explore in fitting in this model
    const float global_speed = params(0);
    const float global_coupling = params(1);

    // regular constants
    const float omega = 60.0 * 2.0 * 3.1415927 / 1e3;

    // coupling constants, coupling itself is hardcoded in kernel
    const float a = 1;

    // coupling parameters
    float c_pop1 = 0.0;

    // derived parameters
    const float rec_n = 1.0f / n_node;
    const float rec_speed_dt = 1.0f / global_speed / (dt);
    const float nsig = sqrt(dt) * sqrt(2.0 * 1e-5);


    curandState crndst;
    curand_init(id * (blockDim.x * gridDim.x * gridDim.y), 0, 0, &crndst);

    float theta = 0.0;

    float dtheta = 0.0;

    //***// This is only initialization of the observable
    for (unsigned int i_node = 0; i_node < n_node; i_node++)
    {
        tavg(i_node) = 0.0f;
        if (i_step == 0){
            state(i_step, i_node) = 0.001;
        }
    }

    //***// This is the loop over time, should stay always the same
    for (unsigned int t = i_step; t < (i_step + n_step); t++)
    {
    //***// This is the loop over nodes, which also should stay the same
        for (int i_node = 0; i_node < n_node; i_node++)
        {
            c_pop1 = 0.0f;

            theta = state((t) % nh, i_node + 0 * n_node);

            // This variable is used to traverse the weights and lengths matrix, which is really just a vector. It is just a displacement. /
            unsigned int i_n = i_node * n_node;

            for (unsigned int j_node = 0; j_node < n_node; j_node++)
            {
                //***// Get the weight of the coupling between node i and node j
                float wij = weights[i_n + j_node]; // nb. not coalesced
                if (wij == 0.0)
                    continue;

                // Get the delay between node i and node j
                unsigned int dij = lengths[i_n + j_node] * rec_speed_dt;

                //***// Get the state of node j which is delayed by dij
                float V_j = state(((t - dij + nh) % nh), j_node + 0 * n_node);

                // Sum it all together using the coupling function. Kuramoto coupling: (postsyn * presyn) == ((a) * (sin(xj - xi))) 
                c_pop1 += wij * a * sin(V_j - V)

## Simulating the result
If the model displays all its features to your whishes, it is time to take her for a spin on a GPU on the JUSUF cluster using PyUnicore.

### Setup PyUnicore

In [14]:
!pip install pyunicore --upgrade
import pyunicore.client as unicore_client
import json
import os
token = clb_oauth.get_token()
tr = unicore_client.Transport(token)
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)
r.site_urls

  Created wheel for pyunicore: filename=pyunicore-0.9.1-py3-none-any.whl size=13319 sha256=e14e0382448d54d31696e1b10d2a070b620e7ac1371565d3a9363d1018e41c10
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vqpdgx2/wheels/df/c3/a4/8ea35f2317f259418ef5f5564921fdffb895a3a4fc2ff7b2b6
Successfully built pyunicore
  Attempting uninstall: pyunicore
    Found existing installation: pyunicore 0.5.7
    Uninstalling pyunicore-0.5.7:
      Successfully uninstalled pyunicore-0.5.7
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


{'RWTH': 'https://unicore.hpc.itc.rwth-aachen.de:8080/RWTH/rest/core',
 'FZJ_JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/FZJ_JURECA/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'irene': 'https://hbp-unicore.ccc.cea.fr/irene/rest/core',
 'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'JUSUF': 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core',
 'JUDAC': 'https://zam2125.zam.kfa-juelich.de:9112/JUDAC/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core',
 'CINECA-GALILEO': 'https://grid.hpc.cineca.it:9111/CINECA-GALILEO/rest/core'}

### Transfer to HPC
Transfer the generated model file to HPC

In [16]:
base_url = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/storages/PROJECT/"
# source_path = os.path.join(os.environ['HOME'], "drive/My Libraries/multiscaleteam/Hackathon Sept20/kuramoto_CUDA.xml")
source_path = os.path.join(os.environ['HOME'], "drive/Shared with groups/RateML TVB/tvb-root/scientific_library/tvb/rateML/generatedModels/kuramoto.c")

# sites = unicore_client.get_sites(tr)
# client = unicore_client.Client(tr, "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/")
# for p in client.get_storages():
#     print (p)

storage = unicore_client.Storage(tr, base_url)
# print(storage.listdir(path))
storage.upload(source_path, destination = "vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/CUDAmodels/Kuramoto.c")

### Job setup

In [17]:
my_job = {}

# executable / application
# arguments for runthingsJusuf are: backend modelname couplings speeds
my_job['Executable'] = "source /p/project/cslns/vandervlag1/cuda_only/tvb-hpc/activate; \
cd /p/project/cslns/vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/example/; \
./runthingsJusuf cuda Kuramoto 32 32;"

# environment vars
# run this on login node, not in batch system
my_job['RunOnLoginNode'] = "true"

my_job['Job type'] = "interactive"

# data stage in - TBD
my_job['Imports'] = []

# data stage out - TBD
my_job['Exports'] = []

# Resources - TBD
my_job['Resources'] = {}

In [18]:
#tr = unicore_client.Transport(token)
HPC_LOC = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core"
# HPC_LOC = "https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core"
site = unicore_client.Client(transport=tr,site_url=HPC_LOC)
#print(json.dumps(site.properties, indent=1))

### Submit Job to JUSUF 

In [23]:
job = site.new_job(job_description=my_job)

### Info about JOB

In [24]:
if(job.is_running()):
    print('Job is running')

job.poll()
print('Job is finished')

Job is running
Job is finished


### Fetch results
Copy the output log from JUSUF to Collab

In [26]:
remote = storage.stat("vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/example/output.out")
remote.download("output.out")

with open("output.out", "r") as f:
    for line in f:
        print (line.rstrip())

model Kuramoto
coupl 32
speed 32


Get .er log

In [27]:
remote = storage.stat("vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/example/error.er")
remote.download("error.er")

with open("error.er", "r") as f:
    for line in f:
        print (line.rstrip())

   INFO  log level set to INFO
   INFO  Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
   INFO  Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
   INFO  NumExpr defaulting to 8 threads.
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory
   attribute tvb.datatypes.time_series.TimeSeries.labels_dimensions = Attr(field_type=<class 'dict'>, default={}, required=True)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory
   attribute tvb.datatypes.projections.ProjectionMatrix.conductances = Attr(field_type=<class 'dict'>, default={'air': 0.0, 'skin': 1.0, 'skull': 0.01, 'brain': 1.0}, required=False)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory
   attribute tvb.simulator.models.oscillator.Generic2dOscillator.state_variable_range = Fin